# Common Tools

In [ ]:
import torch
import numpy as np
import time
from pathlib import Path
import os
import io
from contextlib import redirect_stdout
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchsummary import summary


torch.manual_seed(0)
np.random.seed(0)


log_file_name = str( Path(os.path.abspath(''))  /  "PyTorch.log" )
print(f'log_file_name: {log_file_name}')
print(f'PyTorch version: {torch.__version__}')


#2_1_4_4
#x_train = np.array([[[[0.7262433,0.8173254,0.7680227,0.5581612],[0.2060332,0.5588848,0.9060271,0.4421779],[0.9775497,0.2737045,0.2919063,0.4673147],[0.6326591,0.4695119,0.9821513,0.03036699]]],[[[0.8623701,0.9953471,0.6771811,0.3145918],[0.8169079,0.8480518,0.9919022,0.0326252],[0.699942,0.5262842,0.9340187,0.6876203],[0.5468155,0.08110995,0.1871246,0.4533272]]]], float)
y_2_3 = np.array([[1,0,0],[0,0,1]], float)
#1_1_4_4
#x_train = np.array([[[[0.7262433,0.8173254,0.7680227,0.5581612],[0.2060332,0.5588848,0.9060271,0.4421779],[0.9775497,0.2737045,0.2919063,0.4673147],[0.6326591,0.4695119,0.9821513,0.03036699]]]], float)
#x_train = np.array([[[[1,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]]], float)
#y_train = np.array([[1,0,0]], float)
#1_1_2_2
#x_train = np.array([[[[0.7262433,0.8173254],[0.2060332,0.5588848]]]], float)
#1_2_1_1
#x_train = np.array([[[[0.75]],[[0.25]]]], float)
#1_3_1_1
#x_train = np.array([[[[1.0]],[[0.5]],[[0.0]]]], float)
#1_2_1_1
#x_train = np.array([[[[1.0]],[[0.5]]]], float)
#1_2_2_2
#x_train = np.array([[[[0.0,0.1],[0.2,0.3]],[[0.5,0.6],[0.7,1.0]]]], float)
#2_3_4_5
x_2_3_4_5 = np.array([[[[0.67872983,0.95197606,0.8040681,0.17448357,-0.88190055],[0.17665438,1.2180812,-0.17346638,1.4326493,-0.67888665],[-0.62428117,-0.0980559,0.39797723,-0.09146436,1.4464538],[-1.4088991,1.0871105,1.4860412,0.53154343,-0.55622464]],[[0.9507237,1.0441554,1.4757066,-1.4021244,0.599826],[0.07885243,1.302056,0.56286085,0.1404463,-1.2566701],[-0.9386263,-0.14001845,-0.6084844,1.4656314,0.42809242],[0.7888908,-1.4088172,-0.3569865,-0.47057447,1.3723655]],[[0.0153876385,0.6479175,-1.1431268,-0.67964554,1.2212938],[0.8842968,-0.48851898,-0.12837364,-1.0595249,-0.83605576],[-0.26978016,0.6561805,0.35949084,-0.036095843,-0.9152597],[1.1345744,0.97626954,0.7061925,1.0746577,0.53924704]]],[[[0.3745984,-0.8447797,1.1860874,1.1893194,-1.2402604],[1.0157373,-0.9895715,0.43234587,0.98044324,-0.842437],[1.4999181,-0.05590306,0.45661572,0.65917796,0.92838776],[-1.1971939,0.68728215,0.65535206,-1.1234182,1.2155292]],[[-0.9326286,-0.069570385,0.122385725,-0.52349794,0.51311],[-0.094806775,1.0004907,0.9276114,0.880891,0.79351795],[1.3126212,-0.6150096,0.068355896,0.4901785,-0.5022329],[-0.63983274,0.9618302,-0.8324462,-0.9393852,-1.2944435]],[[0.3738683,0.5791351,0.39118314,1.2829121,-0.83597386],[1.2861229,-1.3004352,1.2003129,0.53551644,-1.2180659],[-1.0527077,-1.1790825,0.32961074,1.3591285,-0.028124375],[-1.0558312,0.53283465,0.20958523,-0.8237906,0.35454643]]]], float)
#1_3_2_2
#x_train = np.array([[[[0.0,0.1],[0.2,0.3]] , [[0.4,0.5],[0.6,0.7]], [[0.8,0.9],[0.95,1.0]] ]], float)
#x_train = np.array([ [1,2] , [3,4]] , float)

#x_train = np.array([[[[0.0,0.0],[0.0,0.0]] , [[0.0,0.0],[0,0]], [[0.0,0],[0.0,1]]]], float)
#x_train = np.array([[ [[0.0,0],[0.0,1]]]], float)


#1_2_2_2
#x_train = np.array([ [[[0.0,0.1],[0.2,0.3]],[[0.5,0.6],[0.7,1.0]]]], float)
#y_train = np.array([[1,0]], float)
#y_train = np.array([[1,0]], float)
#2_2
y_2_2  = np.array([[1,0],[1,0]], float)
#1_1_2_1
#x_train = np.array([[[[0.7262433],[0.2060332]]]], float)
#y_train = np.array([[1,0]], float)
#y_train = np.array([[1,0,0,0,0,0]], float)

#1_1_1_1
#x_train = np.array([[[[1.0]]]], float)
#y_train = np.array([[1,0]], float)

def truncate(s, max_size):
    if len(s)<=max_size:
        return s
    return s[:(max_size//2)]+"..."+s[-(max_size//2):]

def log(tensor, tensor_name = "", display_to_screen = True):
    with torch.no_grad():
        if isinstance(tensor, torch.nn.parameter.Parameter) or isinstance(tensor, torch.Tensor):
            return log(tensor.detach().to('cpu').numpy(), tensor_name, display_to_screen)
        if isinstance(tensor, list):
            for e in tensor:
                log(e, tensor_name, display_to_screen)
            return
        asString = None
        if isinstance(tensor,str):
            asString = tensor
        elif isinstance(tensor,np.ndarray):
            asString = str(tensor_name) + " " + str(tensor.shape) + "\n" + truncate(str(tensor.tolist()), 10000)
        else:
            asString = str(tensor)
        with open(log_file_name, "a") as myfile:
            myfile.write(asString + "\n")
        if display_to_screen:
            print(asString)

def print_intermediate_layer_model(layer, layer_name, layer_input: torch.Tensor, display_to_screen) -> torch.Tensor:
    with torch.no_grad():
        try:
            if layer_input is None:
                return None
            if not isinstance(layer, torch.nn.Module):
                raise Exception(f'not a layer {type(layer)}  {layer}')            
            layer_output = layer.forward(layer_input)
            log(layer_output, layer_name, display_to_screen)
            return layer_output
        except Exception as e:  
            #print(e)
            return None

def print_model(model, inputs, display_weights = True, display_layers_output = False, display_layers_grads = False, display_to_screen = False):
    if not isinstance(model, torch.nn.Module):
        raise Exception(f'not a model{type(model)}  {model}')            
    with torch.no_grad():
        layers = list(model.named_modules())[1:]

        layer_input = inputs
        for layerIndex in range(len(layers)):
            if layerIndex>3 and layerIndex <  (len(layers)-3 and False)  :
                continue
            layer_name = layers[layerIndex][0]
            layer = layers[layerIndex][1]
            log("--------------------------------------------------------------------\n-","", display_to_screen)           
            log("layer#"+ str(layerIndex)+": "+layer_name, "", display_to_screen)
            #log(str(layer.get_config()), "get_config:", False)
            if display_weights:
                for p_name, p in layer.named_parameters():
                    log(p, layer_name+"."+p_name, display_to_screen)
            if display_layers_grads:
                for p_name, p in layer.named_parameters():
                    if p.grad is not None:
                        log(p.grad, layer_name+"."+p_name+" grad:", display_to_screen)
            log("-", "", display_to_screen)
            if display_layers_output:
                log("layer#"+ str(layerIndex)+" output:", "", display_to_screen)
                layer_input = print_intermediate_layer_model(layer, layer_name, layer_input, display_to_screen)
        log("--------------------------------------------------------------------", display_to_screen=display_to_screen)

        

        

def Train(model: torch.nn.Module, 
          X_numpy: np.array,
          Y_numpy: np.array,
          lr: float,
          device: str,
          loss_criterion,
          optimizer,
          numEpochs: int,
          batch_size: int|None = None,
          momentum: float = 0.0,
          lambdaL2Regularization: float = 0.0,
         ) -> (float, float):

    # by default, the batch size will proceed all elements at once
    
    if not batch_size or batch_size <= 0:
        batch_size = X_numpy.shape[0]
    
    if os.path.isfile(log_file_name):
        try:
            os.remove(log_file_name)
        except Exception:
            pass
    
    X = torch.tensor(X_numpy, device=device, dtype=torch.float32)
    Y_true = torch.tensor(Y_numpy, device=device, dtype=torch.float32)
    dataset = TensorDataset(X, Y_true) 
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    log( "--------------------------------------------------------------------\n-")
    f = io.StringIO()
    with redirect_stdout(f):
        summary(model, input_size=(X.shape[1:]), device = device)
    model_summary = f.getvalue()
    log(str(model_summary), '', True)
    
    with torch.no_grad():
        Y_pred_before = model.forward(X)
        train_loss_before = loss_criterion(Y_pred_before, Y_true).to('cpu').item()

    log( "-\n--------------------------------------------------------------------\n-", "", False)
    log(X,"X", False)
    log(Y_true,"Y_true", False)
    log("--------------------------------------------------------------------\nmodel before trainig","", False)
    #log( "--------------------------------------------------------------------\nprint_layers(model) before\n-")
    print_model(model, X, display_weights = True, display_layers_output = True, display_to_screen = False)



    print( "-")
    print( "--------------------------------------------------------------------")
    print( "-")
    print( "training for ",numEpochs," epochs...")

    log( "-\n--------------------------------------------------------------------\n-\nmodel after trainig")
    # log( "--------------------------------------------------------------------\nprint_layers(model) after\n-")


    for epoch in range(0,numEpochs):
        for batch_id, (x_batch,y_batch_true) in enumerate(data_loader):
            y_batch_pred = model(x_batch)
            l = loss_criterion(y_batch_pred, y_batch_true) # compute loss
            #with torch.no_grad():
            #    log(f'loss for epoch#{epoch} / batch_id#{batch_id} : {l.cpu().item()}', '', True)
            l.backward() # compute gradients
            optimizer.step() # update weights

            if epoch == (numEpochs-1):
                with torch.no_grad():
                    print_model(model, X, display_weights = False, display_layers_grads = True ,display_layers_output = False, display_to_screen = False)

            optimizer.zero_grad()


    with torch.no_grad():
        Y_pref_after = model.forward(X)
        train_loss_after = loss_criterion(Y_pref_after, Y_true).to('cpu').item()

    print_model(model, X, display_weights = True, display_layers_output = True, display_to_screen = False)

    log("PyTorch numEpochs = " + str(numEpochs))
    # log("TF learningRate = "+str(keras.backend.eval(optim.lr)))
    log("PyTorch lr = " + str(lr))
    log("PyTorch momentum = " + str(momentum))
    log("PyTorch batch_size = " + str(batch_size))
    log("PyTorch lambdaL2Regularization = " + str(lambdaL2Regularization))
    #log("PyTorch use_bias = " + str(use_bias))
    log(Y_pred_before, "PyTorch Y_pred_before", True)
    log("PyTorch loss_before="+str(train_loss_before))
    log(Y_pref_after, "PyTorch Y_pref_after", True)
    log("PyTorch loss_after="+str(train_loss_after))
    return train_loss_before, train_loss_after
        

# Compute Gradient in PyTorch

In [ ]:
loss = torch.nn.MSELoss()
input = torch.tensor( [[0.01995766907930374, 0.05039015784859657, 0.04157894104719162]], requires_grad=True)
target = torch.tensor([[1.0, 0, 0]])
output = loss(input, target)
output.backward()
input.grad

# Compute Gradient in TensorFlow

In [ ]:
import tensorflow as tf
# Define your predictions and ground truth
predictions = tf.constant([[0.01995766907930374, 0.05039015784859657, 0.04157894104719162]])
ground_truth = tf.constant([[1.0, 0, 0]])

# Compute the Mean Squared Error (MSE) loss
mse_loss = tf.reduce_mean(tf.square(predictions - ground_truth))

# Use GradientTape to compute the gradient
with tf.GradientTape() as tape:
    # Watch the variables we want to compute gradients with respect to
    tape.watch(predictions)
    tape.watch(ground_truth)
    
    # Compute the loss
    loss = tf.reduce_mean(tf.square(predictions - ground_truth))

# Compute the gradient of the loss with respect to the predictions
gradients = tape.gradient(loss, predictions)
gradients

# TestParallelRunWithPyTorch_Mse

In [ ]:
import torch
import numpy as np

class TestParallelRunWithPyTorch_Mse(torch.nn.Module):

    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        np.random.seed(0)
        self.dense = torch.nn.Linear(5, 3)
        torch.nn.init.zeros_(self.dense.bias)
        self.relu = torch.nn.ReLU()
        self.flatten_dense_1 = torch.nn.Flatten(1)
        self.dense_1 = torch.nn.Linear(3*4*3, 3)
        torch.nn.init.zeros_(self.dense_1.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y_dense = self.dense(x)
        y_relu = self.relu(y_dense)
        y_flatten_dense_1 = self.flatten_dense_1(y_relu)
        y_dense_1 = self.dense_1(y_flatten_dense_1)
        return y_dense_1

lr = 0.1
momentum = 0.9
device = 'cuda'
# lambdaL2Regularization = 0.00;
model = TestParallelRunWithPyTorch_Mse().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum = momentum)

loss_before, loss_after = Train(model, 
      x_2_3_4_5,
      y_2_3, 
      lr = lr, 
      device = device,
      loss_criterion = torch.nn.MSELoss(), 
      optimizer = optimizer, 
      numEpochs = 10, 
      momentum = momentum
      )

assert abs(loss_before-0.2612681984901428) < 1e-6
assert abs(loss_after-0.006385428365319967) < 1e-6


# TestParallelRunWithPyTorch_Convolution

In [ ]:


import torch
import numpy as np

'''
class Multiply(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,  mainMatrix : torch.Tensor, diagonalMatrix: torch.Tensor):
        
        print(f'in multply with mainMatrix.shape ={mainMatrix.shape} anddiagonalMatrix.shape = {diagonalMatrix.shape}')
        result = mainMatrix* diagonalMatrix
        print(f'in multply with result.shape ={result.shape}')
        
        return result
'''
    
import torch
import numpy as np

class TestParallelRunWithPyTorch_Convolution(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        np.random.seed(0)
        self.conv2d = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=(1,1), stride=1, padding='same', bias=True)
        torch.nn.init.zeros_(self.conv2d.bias)
        self.conv2d_1 = torch.nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(1,1), stride=1, padding='same', bias=True)
        torch.nn.init.zeros_(self.conv2d_1.bias)
        self.average_pooling2d_ = torch.nn.AvgPool2d(kernel_size=(4,5) )
        self.flatten = torch.nn.Flatten()
        self.dense = torch.nn.Linear(in_features=40, out_features=2, bias=True)
        torch.nn.init.zeros_(self.dense.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y_conv2d = self.conv2d(x)
        y_conv2d_1 = self.conv2d_1(y_conv2d)
        y_average_pooling2d_ = self.average_pooling2d_(y_conv2d_1)
        y_multiply = y_conv2d * y_average_pooling2d_
        y_flatten = self.flatten(y_multiply)
        y_dense = self.dense(y_flatten)
        return y_dense



lr = 0.01
momentum = 0.9
device = 'cuda'
# lambdaL2Regularization = 0.00;
model = TestParallelRunWithPyTorch_Convolution().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum = momentum)

loss_before, loss_after = Train(model, 
      x_2_3_4_5,
      y_2_2, 
      lr = lr, 
      device = device,
      loss_criterion = torch.nn.CrossEntropyLoss(), 
      optimizer = optimizer, 
      numEpochs = 10, 
      momentum = momentum
      )    

assert abs(loss_before-0.6883244514465332) < 1e-6
assert abs(loss_after-0.5018750429153442) < 1e-6
